<a href="https://colab.research.google.com/github/erinijapranckeviciene/MLdata/blob/main/NN_experiments_with_isuic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# isuic data 

In [1]:
import pandas as pd
# read the data
url="https://raw.githubusercontent.com/erinijapranckeviciene/MLdata/main/Kaggle-data-suic.csv"
data = pd.read_csv(url)

In [2]:
print(data.columns)

Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k pop', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation'],
      dtype='object')


Clean the data , change col names

In [4]:
isuic=data.drop(['suicides/100k pop', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'generation', 'population'], axis=1)
print(isuic.columns)



Index(['country', 'year', 'sex', 'age', 'suicides_no', 'gdp_per_capita ($)'], dtype='object')


Describe data

In [5]:
isuic.describe(include=['int','int64','float','object'])

,country,year,sex,age,suicides_no,gdp_per_capita ($)
count,27820,27820.000000,27820,27820,27820.000000,27820.000000
unique,101,NaN,2,6,NaN,NaN
top,Austria,NaN,female,75+ years,NaN,NaN
freq,382,NaN,13910,4642,NaN,NaN
mean,NaN,2001.258375,NaN,NaN,242.574407,16866.464414
std,NaN,8.469055,NaN,NaN,902.047917,18887.576472
min,NaN,1985.000000,NaN,NaN,0.000000,251.000000
25%,NaN,1995.000000,NaN,NaN,3.000000,3447.000000
50%,NaN,2002.000000,NaN,NaN,25.000000,9372.000000
75%,NaN,2008.000000,NaN,NaN,131.000000,24874.000000


In [12]:
Y_country=isuic['country']
X_country=isuic.drop(['country'], axis=1)

Y_year=isuic['year']
X_year=isuic.drop(['year'], axis=1)

Y_sex=isuic['sex']
X_sex=isuic.drop(['sex'], axis=1)

Y_age=isuic['age']
X_age=isuic.drop(['age'], axis=1)

Y_suicno=isuic['suicides_no']
X_suicno=isuic.drop(['suicides_no'], axis=1)


In [13]:
X_country_dummies=pd.get_dummies(X_country)
#list(isuic_male_reduced_dummies.columns)
print(X_country_dummies.shape)

(27820, 11)


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_country_dummies, Y_country, random_state=1)
print("Training features/target:", X_train.shape, y_train.shape)
print("Testing features/target:", X_test.shape, y_test.shape)

Training features/target: (20865, 11) (20865,)
Testing features/target: (6955, 11) (6955,)


In [24]:
import numpy as np
np.count_nonzero(np.isnan(X_train))

0

In [26]:
from tensorflow import keras
encoder = keras.models.Sequential([keras.layers.Dense(2,  input_shape=[11])])
decoder = keras.models.Sequential([keras.layers.Dense(11, input_shape=[2])])
autoencoder = keras.models.Sequential([encoder, decoder])
autoencoder.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=0.1))

In [28]:
autoencoder.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_10 (Sequential)   (None, 2)                 24        
_________________________________________________________________
sequential_11 (Sequential)   (None, 11)                33        
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [27]:
history = autoencoder.fit(X_train, X_train, epochs=20)
codings = encoder.predict(X_train)

Epoch 1/20
653/653 [==============================] - 1s 819us/step - loss: nan
Epoch 2/20
653/653 [==============================] - 1s 804us/step - loss: nan
Epoch 3/20
653/653 [==============================] - 1s 865us/step - loss: nan
Epoch 4/20
653/653 [==============================] - 1s 845us/step - loss: nan
Epoch 5/20
653/653 [==============================] - 1s 845us/step - loss: nan
Epoch 6/20
653/653 [==============================] - 1s 874us/step - loss: nan
Epoch 7/20
653/653 [==============================] - 1s 860us/step - loss: nan
Epoch 8/20
653/653 [==============================] - 1s 867us/step - loss: nan
Epoch 9/20
653/653 [==============================] - 1s 860us/step - loss: nan
Epoch 10/20
653/653 [==============================] - 1s 886us/step - loss: nan
Epoch 11/20
653/653 [==============================] - 1s 869us/step - loss: nan
Epoch 12/20
653/653 [==============================] - 1s 842us/step - loss: nan
Epoch 13/20
653/653 [================

In [25]:
print(codings)

[[nan nan]
 [nan nan]
 [nan nan]
 ...
 [nan nan]
 [nan nan]
 [nan nan]]


In [16]:
stacked_encoder = keras.models.Sequential([keras.layers.Flatten(input_shape=[11, 1]), keras.layers.Dense(100, activation="selu"), keras.layers.Dense(5, activation="selu"),])
